In [15]:
import re
import time
import json
import csv
import random
import requests
import pprint
import pickle
import unidecode
import numpy as np
import pandas as pd
from random import sample
import reverse_geocode
import seaborn as sns
from sklearn import metrics
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import chisquare
from collections import Counter, defaultdict
from sklearn.metrics import cohen_kappa_score

In [3]:
Data_Root = '/home/haopeng/Data/Promotion/'
data_root = '/home/haopeng/Data/Retraction/'

In [4]:
def yield_one_line(filename, delimiter = '\t', quote = csv.QUOTE_NONE):
    '''a generator which produce one line of a given file'''
    with open(filename, 'r') as file:
        print('processing %s...' %(filename))
        reader = csv.reader(file, delimiter=delimiter, quoting=quote)
        for row in reader:
            yield row

In [ ]:
# type_cn = defaultdict(int,
#             {'article': 18266091,
#              'chapter': 6255269,
#              'book': 1490369,
#              'clinical_trial_study_record': 49758,
#              'news': 130360,
#              'dataset': 30883,
#              'Journal Article': 15,
#              'JOURNAL ARTICLE': 8,
#              'Editorial': 1})

In [6]:
plat_cate = {
 'blogs': 'blog',
 'news': 'news_media',
 'patent': 'knowledge_repo',
 'f1000': 'knowledge_repo',
 'peer_reviews': 'knowledge_repo',
 'wikipedia': 'knowledge_repo',
 'q&a': 'knowledge_repo',
 'facebook': 'social_media',
 'googleplus': 'social_media',
 'linkedin': 'social_media',
 'pinterest': 'social_media',
 'reddit': 'social_media',
 'twitter': 'social_media',
 'video': 'social_media'
}

In [7]:
tweets_cn = 0
social_media_cn = 0
posts_cn = 0
cn_lines = 0

with open(data_root + 'merged.txt', 'r') as ofile:
    for row in ofile:
        cn_lines += 1
        if cn_lines % 1000000 == 0:
            print('processed %d lines...'%cn_lines)
        if '}{"altmetric_id"' in row:
            row = row.replace('}{"altmetric_id"', '}\n{"altmetric_id"')
        # this way works for both good and bad lines.
        records = row.split('\n')
        for record in records:
            if record != '':
                paper = json.loads(record)
                citation = paper['citation']
                doi = ''
                if 'doi' in citation:
                    doi = citation['doi']
                # focus on research articles published in 2018
                if doi != '':
                    for plat in paper['posts']:
                        if plat == 'twitter':
                            tweets_cn += len(paper['posts'][plat])
                        if plat in plat_cate and plat_cate[plat] == 'social_media':
                            social_media_cn += len(paper['posts'][plat])
                        posts_cn += len(paper['posts'][plat])

processed 1000000 lines...
processed 2000000 lines...
processed 3000000 lines...
processed 4000000 lines...
processed 5000000 lines...
processed 6000000 lines...
processed 7000000 lines...
processed 8000000 lines...
processed 9000000 lines...
processed 10000000 lines...
processed 11000000 lines...
processed 12000000 lines...
processed 13000000 lines...
processed 14000000 lines...
processed 15000000 lines...
processed 16000000 lines...
processed 17000000 lines...
processed 18000000 lines...
processed 19000000 lines...
processed 20000000 lines...
processed 21000000 lines...
processed 22000000 lines...
processed 23000000 lines...
processed 24000000 lines...
processed 25000000 lines...


In [8]:
tweets_cn / social_media_cn

0.9283170609434604

In [9]:
tweets_cn / posts_cn

0.7389792916366573

In [10]:
social_media_cn / posts_cn

0.7960419157713458

In [37]:
paper_li = {}
cn_lines = 0

with open(data_root + 'merged.txt', 'r') as ofile:
    for row in ofile:
        cn_lines += 1
        if cn_lines % 1000000 == 0:
            print('processed %d lines...'%cn_lines)
        if '}{"altmetric_id"' in row:
            row = row.replace('}{"altmetric_id"', '}\n{"altmetric_id"')
        # this way works for both good and bad lines.
        records = row.split('\n')
        for record in records:
            if record != '':
                paper = json.loads(record)
                citation = paper['citation']
                pubdate = ''
                if 'pubdate' in citation:
                    pubdate = citation['pubdate']
                elif 'epubdate' in citation:
                    pubdate = citation['epubdate']
                dtype = ''
                if 'type' in citation:
                    dtype = citation['type']
                doi = ''
                if 'doi' in citation:
                    doi = citation['doi']
                # focus on research articles published in 2018
                if pubdate != '' and pubdate[:4] == '2018' and dtype == 'article' and doi != '':
                    # authors = []
                    # if 'authors' in citation:
                    #    authors = citation['authors']
                    jnl = ''
                    if 'journal' in citation:
                        jnl = citation['journal']
                    scopus_subjects = []
                    if 'scopus_subjects' in citation:
                        scopus_subjects = citation['scopus_subjects']
                    tweets = []
                    if 'twitter' in paper['posts']:
                        tweets = paper['posts']['twitter']
                    # could add pub year later here.
                    pp = {'journal': jnl, 'scopus_subjects': scopus_subjects, 'tweets': tweets}
                    paper_li[doi] = pp

processed 1000000 lines...
processed 2000000 lines...
processed 3000000 lines...
processed 4000000 lines...
processed 5000000 lines...
processed 6000000 lines...
processed 7000000 lines...
processed 8000000 lines...
processed 9000000 lines...
processed 10000000 lines...
processed 11000000 lines...
processed 12000000 lines...
processed 13000000 lines...
processed 14000000 lines...
processed 15000000 lines...
processed 16000000 lines...
processed 17000000 lines...
processed 18000000 lines...
processed 19000000 lines...
processed 20000000 lines...
processed 21000000 lines...
processed 22000000 lines...
processed 23000000 lines...
processed 24000000 lines...
processed 25000000 lines...


In [43]:
len(paper_li)

1218710

In [41]:
with open(Data_Root+'papers_2018.json', 'w') as ofile:
    for doi in paper_li:
        res = {'doi': doi, 'data': paper_li[doi]}
        ofile.write(json.dumps(res)+'\n')

In [6]:
paper_li = {}
with open(Data_Root+'papers_2018.json', 'r') as ifile:
    for line in ifile:
        res = json.loads(line)
        paper_li[res['doi']] = res['data']

In [11]:
i = 0
for doi in paper_li:
    paper = paper_li[doi]
    if len(paper['scopus_subjects']) == 0:
        i += 1

In [13]:
# no discipline info.
i / len(paper_li)

0.3490059160916051

In [15]:
discs_broad = set(['Social Sciences', 'Life Sciences', 'Health Sciences', 'Physical Sciences', 'General'])

# every paper has one of the five broad disciplines.
# every paper has subfiels and their associated broad disciplines, so there is no need to map sub to broad separately.
i = 0
for doi in paper_li:
    paper = paper_li[doi]
    if len(paper['scopus_subjects']) > 0:
        for sub in paper['scopus_subjects']:
            if sub in discs_broad:
                break
        else:
            if i < 10:
                print(paper['scopus_subjects'])
            i += 1

In [100]:
# ['General']
# ['Physical Sciences', 'General', 'Mathematics', 'Engineering']
sub_cn = defaultdict(int)
for doi in paper_li:
    paper = paper_li[doi]
    for sub in paper['scopus_subjects']:
        sub_cn[sub] += 1

In [101]:
# num. of papers with a specific keyword
sorted(sub_cn.items(), key=lambda x: x[1]*-1)

[('Health Sciences', 358843),
 ('Medicine', 338170),
 ('Physical Sciences', 259634),
 ('Life Sciences', 254596),
 ('Biochemistry, Genetics and Molecular Biology', 145828),
 ('Social Sciences', 121705),
 ('Agricultural and Biological Sciences', 78501),
 ('Chemistry', 73449),
 ('Physics and Astronomy', 70245),
 ('Environmental Science', 51201),
 ('Engineering', 50395),
 ('Materials Science', 42561),
 ('Earth and Planetary Sciences', 37551),
 ('Neuroscience', 34005),
 ('Psychology', 32549),
 ('Immunology and Microbiology', 31777),
 ('Pharmacology, Toxicology and Pharmaceutics', 29762),
 ('Chemical Engineering', 28420),
 ('Computer Science', 28171),
 ('Mathematics', 23106),
 ('Nursing', 21877),
 ('Arts and Humanities', 16630),
 ('Business, Management and Accounting', 16179),
 ('Health Professions', 14187),
 ('Economics, Econometrics and Finance', 13121),
 ('Energy', 11190),
 ('General', 9059),
 ('Veterinary', 8179),
 ('Decision Sciences', 5792),
 ('Dentistry', 3966)]

In [602]:
len(sub_cn)

30

In [6]:
# Social Sciecnes is created in the subfields because there is also a `social sciences` subfield category.
discs = {'Life Sciences': 0, 'Health Sciences': 1, 'Physical Sciences': 2}

# 26 subfields + 1 General
fields = {'Social Sciences': 'Social Sciences',
 'Materials Science': 'Physical Sciences',
 'Engineering': 'Physical Sciences',
 'Chemistry': 'Physical Sciences',
 'Biochemistry, Genetics and Molecular Biology': 'Life Sciences',
 'Medicine': 'Health Sciences',
 'Nursing': 'Health Sciences',
 'Agricultural and Biological Sciences': 'Life Sciences',
 'Pharmacology, Toxicology and Pharmaceutics': 'Life Sciences',
 'Neuroscience': 'Life Sciences',
 'Business, Management and Accounting': 'Social Sciences',
 'Economics, Econometrics and Finance': 'Social Sciences',
 'Chemical Engineering': 'Physical Sciences',
 'Physics and Astronomy': 'Physical Sciences',
 'Computer Science': 'Physical Sciences',
 'Decision Sciences': 'Social Sciences',
 'Health Professions': 'Health Sciences',
 'Psychology': 'Social Sciences',
 'Immunology and Microbiology': 'Life Sciences',
 'Dentistry': 'Health Sciences',
 'Earth and Planetary Sciences': 'Physical Sciences',
 'Environmental Science': 'Physical Sciences',
 'Mathematics': 'Physical Sciences',
 'Arts and Humanities': 'Social Sciences',
 'Energy': 'Physical Sciences',
 'Veterinary': 'Health Sciences',
 'General': 'General'}

field_ix = {fie: ix for ix, fie in enumerate(list(fields.keys()))}

In [93]:
len(fields)

27

In [56]:
with open(Data_Root+'alt_dois.pickle', 'wb') as ofile:
    pickle.dump(list(paper_li.keys()), ofile)

In [ ]:
# 85411606 tids in total, which contains 14412 tids that are not numeric string.
# e.g., '1.5558470649316E+17', '1.5464891076012E+17', '1.6227338710274E+17'

In [26]:
# An example tweet structure

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Feb 17 09:01:16 +0000 2017',
 'display_text_range': [0, 116],
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'bit.ly/2kqg0ym',
                        'expanded_url': 'http://bit.ly/2kqg0ym',
                        'indices': [77, 100],
                        'url': 'https://t.co/PcpdHjw86p'}],
              'user_mentions': [{'id': 96741105,
                                 'id_str': '96741105',
                                 'indices': [0, 15],
                                 'name': 'Newcastle University',
                                 'screen_name': 'UniofNewcastle'},
                                {'id': 96741105,
                                 'id_str': '96741105',
                                 'indices': [101, 116],
                                 'name': 'Newcastle University',
                                 'screen_name': 'UniofNewcastle'}]},
 'f

In [ ]:
tid_name_sname_org = {}
tid_name_sname_retweet = {}

# with open(data_root+'second_match/tweets_clean.json', 'r') as ifile:
with open(Data_Root+'tweets_v2.json', 'r') as ifile:
    for line in ifile:
        line = json.loads(line)
        tid = line['id_str'] # line['created_at']
        if 'retweeted_status' in line:
            tid_name_sname_retweet[tid] = (line['user']['name'], line['user']['screen_name'])
        else:
            # pprint.pprint(line)
            tid_name_sname_org[tid] = (line['user']['name'], line['user']['screen_name'])
            # tid_date_all[tid] = datetime.strptime(tdate, '%a %b %d %H:%M:%S +0000 %Y')#.strftime("%Y-%m-%d")
        
with open(Data_Root+'tweets_v2_non_full_text.json', 'r') as ifile:
    for line in ifile:
        line = json.loads(line)
        tid = line['id_str']
        if 'retweeted_status' in line:
            tid_name_sname_retweet[tid] = (line['user']['name'], line['user']['screen_name'])
        else:
            tid_name_sname_org[tid] = (line['user']['name'], line['user']['screen_name'])

In [19]:
len(tid_name_sname_org)

34362660


In [66]:
len(tid_name_sname_retweet)

43134644

In [67]:
with open(Data_Root+'tid_name_sname.json', 'w') as ofile:
    for tid in tid_name_sname_org:
        res = {'tid': tid, 'type': 'org', 'data': tid_name_sname_org[tid]}
        ofile.write(json.dumps(res)+'\n')
    for tid in tid_name_sname_retweet:
        res = {'tid': tid, 'type': 're', 'data': tid_name_sname_retweet[tid]}
        ofile.write(json.dumps(res)+'\n')

In [20]:
tid_name_sname_org = {}
tid_name_sname_retweet = {}

with open(Data_Root+'tid_name_sname.json', 'r') as ifile:
    for line in ifile:
        res = json.loads(line)
        if res['type'] == 'org':
            tid_name_sname_org[res['tid']] = res['data']
        else:
            tid_name_sname_retweet[res['tid']] = res['data']

In [ ]:
doi_author_list = defaultdict(list)

with open(Data_Root+'dois_authors_mag.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        doi, authors = row['doi'], row['authors']
        uniq_authors = defaultdict(lambda: ['', [], ''])
        for aid, affi_id, seq, name in authors:
            seq = int(seq)
            uniq_authors[seq][0] = aid
            uniq_authors[seq][1].append(affi_id)
            # normalized author name.
            uniq_authors[seq][2] = unidecode.unidecode(name).strip()
        # could be [1, 2, 4, 6]; [2, 3]
        uniq_seqs = sorted(uniq_authors)
        # must be: 1, 2, ..., max
        if uniq_seqs == list(range(1, len(uniq_seqs)+1)):
            last_seq = len(uniq_seqs)
            if last_seq == 1:
                aid, affi_ids, name = uniq_authors[last_seq]
                affis = '|'.join(affi_ids)
                doi_author_list[doi].append([aid, affis, last_seq, name, 'solo_author'])
            else:
                for seq in uniq_authors:
                    aid, affi_ids, name = uniq_authors[seq]
                    affis = '|'.join(affi_ids)
                    if seq == 1:
                        doi_author_list[doi].append([aid, affis, seq, name, 'first_position'])
                    elif seq == last_seq:
                        doi_author_list[doi].append([aid, affis, seq, name, 'last_position'])
                    else:
                        doi_author_list[doi].append([aid, affis, seq, name, 'middle_position'])

In [ ]:
len(doi_author_list)

In [77]:
doi_author_list['10.1001/amajethics.2018.1003']

[['2899491243', '', 1, 'Annie J. Tsay', 'solo_author']]

In [199]:
author_names = set()
for doi in doi_author_list:
    for aut in doi_author_list[doi]:
        author_name = aut[3]
        author_names.add(author_name)

In [200]:
len(author_names)

3081657

In [203]:
with open(Data_Root+'author_names.txt', 'w') as ofile:
    for author_name in author_names:
        ofile.write(author_name+'\n')

In [ ]:
# predict gender based on name using Ford et al.
# see `predict_gender.ipynb` (run code using Python 2.7)
name_gender = {}
with open(Data_Root+'name_gender.txt', 'r') as ifile:
    for line in ifile:
        # get rid of '\n' at the end of each line
        line = line.strip()
        name, gender = line.split('||')
        name_gender[name] = gender

In [ ]:
len(name_gender)

In [31]:
aid_metric = {}
with open(Data_Root+'aids_metric_mag.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        aid, metric = row['aid'], row['metric']
        aid_metric[aid] = [int(num) for num in metric]

In [32]:
len(aid_metric)

3335676

In [16]:
aid_year_paper_count = {}
with open(Data_Root+'aid_year_paper_count.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        aid, paper_cn = row['aid'], row['paper_cn']
        aid_year_paper_count[aid] = paper_cn

In [17]:
len(aid_year_paper_count)

3334118

In [19]:
aid_year_paper_count['2704166667']

{'2015': 7,
 '2010': 2,
 '2013': 1,
 '2011': 1,
 '2014': 4,
 '2008': 1,
 '2005': 2,
 '2003': 1,
 '2012': 3,
 '2016': 8,
 '2017': 7,
 '2018': 16}

In [34]:
aid_cites_upto_2017 = {}

with open(Data_Root+'aid_cites_upto_2017.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        aid, cites = row['aid'], row['cites']
        aid_cites_upto_2017[aid] = cites

In [35]:
len(aid_cites_upto_2017)

3335676

In [31]:
def get_author_name(name):
    words = name.split()
    if len(words) <= 1:
        payload = 'invalid'
    else:
        payload = {'Fname': words[0], 'Lname': words[-1]}
    return payload

def check_match_author(tname_tuple, name_tupe):
    hname, sname = tname_tuple
    hname = unidecode.unidecode(hname).lower()
    sname = unidecode.unidecode(sname).lower()
    htokens, stokens = [hname], [sname]
    if ' ' in hname:
        htokens = hname.split(' ')
    elif '_' in hname:
        htokens = hname.split('_')
    if ' ' in sname:
        stokens = sname.split(' ')
    elif '_' in sname:
        stokens = sname.split('_')
    tokens = htokens + stokens
    fname, lname = name_tupe['Fname'], name_tupe['Lname']
    fname = fname.lower()
    lname = lname.lower()
    if len(tokens) > 2:
        # don't use first name if it is initial. e.g., True 3 M. B. Mendonca (Plecoforum.nl)
        if (len(fname) == 1) or (len(fname) == 2 and fname[-1] == '.'):
            if lname in tokens:
                return True
        else:
            if (fname in tokens) or (lname in tokens):
                return True
        return False
    # both user name and screen name are single-token string: use containment matching only for names with >= 4 chars.
    else:
        if len(fname) >= 4:
            if (fname in hname) or (fname in sname):
                return True
        if len(lname) >= 4:
            if (lname in hname) or (lname in sname):
                return True
        return False

### Evaluate the simple matching heuristic

In [154]:
# valid_dois = []

# for doi in paper_li:
#     paper = paper_li[doi]
#     mtids = [tweet['tweet_id'] for tweet in paper['tweets'] if tweet['tweet_id'] in tid_name_sname]
#     # at least one tweet
#     if len(mtids) >= 1:
#         valid_dois.append(doi)

In [282]:
len(valid_dois)

946289

In [573]:
dois = sample(valid_dois, 100)

In [580]:
cn = 0
p_cn = 0
tp_cn = 0
# manual_true = []
for doi in dois:
    if doi in doi_author_list:
        print('\nFound authors for this paper, checking each author. DOI: %s'%doi)
        paper = paper_li[doi]
        mtids = [tweet['tweet_id'] for tweet in paper['tweets'] if tweet['tweet_id'] in tid_name_sname_org]
        for tid in mtids:
            print(tid, tid_name_sname_org[tid])
        for author in doi_author_list[doi]:
            aid, affis, seq, author_name, pos = author
            name_tupe = get_author_name(author_name)
            if type(name_tupe) == dict:
                cn += 1
                self_promo = False
                mat = ''
                matches = []
                for tid in mtids:
                    tname_tupe = tid_name_sname_org[tid]
                    if check_match_author(tname_tupe, name_tupe):
                        matches.append(tid_name_sname_org[tid][0])
                if len(matches) >= 1:
                    self_promo = True
                    if len(matches) == 1:
                        mat = matches[0]
                    else:
                        name_scores = {}
                        for tw_name in matches:
                            name_scores[tw_name] = fuzz.ratio(tw_name, author_name)
                        mat = sorted(name_scores.items(), key=lambda x: x[1], reverse=True)[0][0]
                if self_promo:
                    p_cn += 1
                    res = '  ' + str(self_promo) + '\t' + str(seq) + ' ' + author_name + ' (%s)'%mat
                    if (doi, author_name) in manual_true:
                        tp_cn += 1
                    # manual_true.append((doi, author_name))
                else:
                    res = '  ' + str(self_promo) + '\t' + str(seq) + ' ' + author_name
                print(res)
                            
            else:
                print('\tSkip author due to invalid name for this paper.')
    else:
        print('\nNo author info for this paper, skip.')


Found authors for this paper, checking each author. DOI: 10.4269/ajtmh.16-0978
945971792690208768 ('Bill Tropical Health', 'bbbrieger')
  False	2 Harihar Sahoo
  False	4 Kailash Chandra Das
  False	1 Dipti Govil
  False	3 Sarang P. Pedgaonkar
  False	5 Hemkhothang Lhungdim

Found authors for this paper, checking each author. DOI: 10.1016/j.celrep.2018.09.003
1047939680543494144 ('micropapers', 'micro_papers')
1048660430309220352 ('Immunology of Infectious Disease News', 'infdisease_news')
1049045701664149504 ('Atanas G. Atanasov', '_atanas_')
1049400372035506176 ('Neural Cell News', 'NeuralCell')
1052257909705531392 ('Cell Reports', 'CellReports')
1133006149144514560 ('Oliver Härschnitz', 'harschnitz')
  False	16 Martin Stangel
  False	15 Frank Bradke
  False	17 Ulrich Kalinke
  False	11 Chittappen K. Prajeeth
  False	3 Elena Grabski
  False	1 Chintan Chhatbar
  False	4 Katharina Borst
  False	2 Claudia N. Detje
  False	12 Viktoria Gudi
  False	13 Michael A. Klein
  False	5 Julia Span

In [578]:
# first iteration
manual_true = [
 ('10.1039/c7bm01156c', 'Gordon G Wallace'),
 ('10.3828/bj.2018.5', 'Frank Lawton'),
 ('10.1016/j.conctc.2018.04.002', 'Nayanjeet Chaudhury'),
 ('10.1186/s12862-018-1204-3', 'Anna Bruniche-Olsen'),
 ('10.1021/acs.joc.8b01766', 'Phillip P. Sharp'),
 ('10.1073/pnas.1801692115', 'Claire A. Runge'),
 ('10.1073/pnas.1801692115', 'Halley E. Froehlich'),
 ('10.1016/j.ijdrr.2018.03.017', 'Prasoon Singh'),
 ('10.1016/j.jconrel.2018.08.018', 'Zheng Chai'),  # ('Zheng', 'chaizheng20015')
 ('10.1016/j.jconrel.2018.08.018', 'Xintao Zhang'), # ('Xintao', 'xintaoz2000')
 ('10.1017/rep.2017.38', 'Christopher C. Towler'),
 ('10.1101/304600', 'Sean M Murray'),
 ('10.1007/s11136-018-1898-0', 'Douglas P. Gross'),
 ('10.1162/neco_a_01138', 'Ian H. Stevenson'),
 ('10.1177/1465116518793707', 'Jose Luis Castro-Montero'),
 ('10.1177/1465116518793707', 'Edwin Alblas'),
 ('10.3389/fpsyg.2018.01320', 'Michelle K. Ryan'),
 ('10.3389/fpsyg.2018.01320', 'Thekla Morgenroth'),
 ('10.1002/anie.201806221', 'Rafael Luque'),
 ('10.1021/acsnano.8b06914', 'Martin Pumera'),
 ('10.1103/physreva.97.022312', 'Tadashi Kadowaki')
]

# second iteration
manual_true = [
 ('10.1038/s41560-018-0156-6', 'Rob Bellamy'),
 ('10.1016/j.peptides.2017.12.002', 'Catherine Arden'),
 ('10.1080/09636412.2017.1416813', 'Tricia Bacon'),
 ('10.1016/j.paid.2016.12.013', 'Jennifer L. Bryan'),
 ('10.1097/jxx.0000000000000023', 'Heather M. Brom'),
 ('10.1016/j.tube.2018.08.004', 'Sean Ekins'),
 ('10.1088/2057-1976/aabab2', 'Moeava Tehei'),
 ('10.1073/pnas.1801759115', 'Karl E. Kadler'),
 ('10.1073/pnas.1801759115', 'Ching-Yan Chloe Yeung'),
 ('10.1038/s41598-018-32252-1', 'Martin Hromada'),
 ('10.1038/s41598-018-32252-1', 'Peter Mikula'),
 ('10.1111/tpj.13801', 'Ralf Reski'),
 ('10.1111/tpj.13801', 'Stefan A. Rensing'),
 ('10.1111/tpj.13801', 'Stuart F. McDaniel'),
 ('10.1007/s11528-018-0310-9', 'Heather Toomey Zimmerman'),
 ('10.17605/osf.io/k495x', 'Aidan J Horner'),
 ('10.1177/1079063216639591', 'Steven M. Gillespie'),
 ('10.1007/s10531-018-1582-2', 'Barry W. Brook'),
 ('10.1371/journal.pcbi.1005958', 'Xavier Didelot'),
 ('10.1080/01402382.2017.1332328', 'Joost van Spanje')
]

manual_true = [
 ('10.1016/j.sexol.2018.05.004', 'R Rossi'),
 ('10.1152/ajprenal.00566.2017', 'David O. Bates'),
 ('10.4049/jimmunol.1701755', 'John R. Lukens'),
 ('10.1111/mmi.13872', 'Matthew Joseph Powers'),
 ('10.1093/infdis/jiy347', 'Alison K Criss'),
 ('10.1038/s41437-017-0019-0', 'Emily E. Puckett'),
 ('10.1002/jlb.6ma1117-451r', 'Carolyn A. de Graaf'),
 ('10.1007/s10278-017-0013-3', 'Curtis P. Langlotz'),
 ('10.1007/s10278-017-0013-3', 'Falgun H. Chokshi'),
 ('10.1080/0267257x.2018.1484791', 'Stephen Dunne'),
 ('10.1038/s41366-018-0196-9', 'Michelle A. Morris'),
 ('10.7717/peerj.5194', 'David C. Evans'),
 ('10.7717/peerj.5194', 'Michael J. Ryan'),
 ('10.7717/peerj.5924', 'Ivar Folstad'),
 ('10.3389/fcimb.2018.00197', 'Emma Carmelo'),
 ('10.1016/j.pmr.2018.01.004', 'Phoebe Runciman'),
 ('10.3389/fnins.2018.00941', 'Thomas Nowotny'),
 ('10.3389/fnins.2018.00941', 'James C Knight'),
 ('10.1039/c8ta04267e', 'Giuliano Giambastiani')
]

manual_true = [
 ('10.3174/ajnr.a5833', 'R. Khangura'),
 ('10.3847/1538-3881/aaa292', 'Joseph E. Rodriguez'),
 ('10.1016/j.cma.2017.08.027', 'Jacobo Ayensa-Jimenez'),
 ('10.1016/j.jmgm.2018.06.005', 'David Ryan Koes'),
 ('10.1093/ejcts/ezx297', 'Douglas George West'),
 ('10.3389/fped.2018.00380', 'Kim D Lu'),
 ('10.1103/physrevlett.120.191103', 'Katelyn Breivik'),
 ('10.1007/s10508-017-0982-9', 'Anne-Rose Robbins'),
 ('10.1080/02691728.2018.1466933', 'Slobodan Perovic'),
 ('10.1038/s41467-018-07571-6', 'Juan Bisquert'),
 ('10.1016/j.bbamcr.2018.04.005', 'Leonardo Mirandola'),
 ('10.1177/0308275x18775202', 'Irene Skovgaard-Smith'),
 ('10.1097/sla.0000000000002195', 'Alejandro Roman-Gonzalez'),
 ('10.1007/s41030-018-0063-9', 'John Thomas'),
 ('10.1007/s41030-018-0063-9', 'Adam Lewis'),
 ('10.1080/13675567.2018.1431211', 'Henrik Sternberg'),
 ('10.1242/jeb.170316', 'Brittney Borowiec'),
 ('10.1177/1073110518766029', 'Jonathan Todres'),
 ('10.1139/apnm-2017-0763', 'Kara Solmundson')
]

In [581]:
len(manual_true)

19

In [582]:
cn

517

In [583]:
p_cn

20

In [584]:
tp_cn

19

First iteration

In [ ]:
# only containment matching
# False predictions:

True 1 Xuecai Ge (HPB_surgery)
True 16 Daniel D. Kaplan (Daniel J Drucker)
True 4 P A Barber (Jesús Porta-Etessam)
True 6 Man Su Kim (metagen human papers)
True 2 Christopher S. Parker (Christopher Federico)
True 1 Songfeng Li (Massimiliano Versace)

False 1 Tadashi Kadowaki (tadakado)

In [ ]:
# only token matching.
# False predictions:

True 16 Daniel D. Kaplan (Daniel J Drucker)
True 2 Christopher S. Parker (Christopher Federico)

False 1 Tadashi Kadowaki (tadakado)
False 1 Zheng Chai (chaizheng20015)
False 2 Xintao Zhang (xintaoz2000)

In [ ]:
# containment matching for single-token handle and screen names, otherwise token matching.
# False predictions:

True 16 Daniel D. Kaplan (Daniel J Drucker)
True 2 Christopher S. Parker (Christopher Federico)

False 1 Tadashi Kadowaki (tadakado)

Second iteration (containment matching for single-token handle and screen names, otherwise token matching.)

In [ ]:
# False predictions:

True 1 Yifei Ma (BioMassSpec)
True 1 Heather M. Brom (Heather Tubbs Cooley, PhD, RN, FAAN) # but it matches ('Heather Brom', 'heather_brom')

Third iteration (matching multiple users)

In [ ]:
# False predictions:

True 13 Yan Shi (fireflyann)
True 3 Xin-Li An (AntibioticResistance)
True 10 Hua-Bin Li (PharmSolidForms)
True 8 Sha Li (PharmSolidForms)
True 2 Michael J. Nooromid (Michael Zenilman)

Fourth iteration (containment matching limited to 4-char)

In [ ]:
# False predictions:

True 1 Peter D. Maskell (Peter “1 Jag”)

### Build regression data

In [38]:
table = str.maketrans(",'&:-", "     ")

def norm_string(aff_name):
    aff_name = aff_name.lower().translate(table)
    aff_name = ' '.join(aff_name.split())
    return aff_name

def get_journal_rank(journal):
    if journal == 'unknown':
        return np.NaN
    else:
        nname = norm_string(journal)
        if nname in journal_impact:
            return journal_impact[nname]
        else:
            return np.NaN
        
def get_journal_cate(journal):
    if journal in top_journals or journal == 'unknown':
        return journal
    else:
        return 'lose journal'

def get_affi_name(affis):
    affi_li = affis.split('|')
    univs = []
    for affi_id in affi_li:
        if affi_id in affi_name:
            univs.append(affi_name[affi_id])
    if len(univs) == 0:
        return 'unknown'
    else:
        return '|'.join(univs)
    
def get_affi_cate(affis):
    affi_li = affis.split('|')
    cate_li = set()
    for affi_id in affi_li:
        # many affi ids in MAG do not have latitude and longitude info for us to infer their country
        if affi_id in affi_country:
            country = affi_country[affi_id]
            if country == 'United States':
                cate_li.add('domestic')
            else:
                cate_li.add('international')
        else:
            cate_li.add('unknown')
    if 'domestic' in cate_li:
        return 'domestic'
    elif 'international' in cate_li:
        return 'international'
    else:
        return 'unknown'
        
def get_affi_rank(affis):
    affi_li = affis.split('|')
    rank_li = []
    for affi_id in affi_li:
        if affi_id in affi_rank:
            rank_li.append(affi_rank[affi_id])
    if len(rank_li) == 0:
        return np.NaN
    else:
        return min(rank_li)

def get_author_rank(aid):
    if aid in aid_metric:
        return aid_metric[aid][0]
    else:
        return np.NaN    

def get_aut_pub_cn(row):
    pub_year, aid = row['pub_year'], row['author_id']
    if aid in aid_year_paper_count:
        total = 0
        pub_year = int(pub_year)
        for year, cn in aid_year_paper_count[aid].items():
            if int(year) <= pub_year:
                total += cn
        return total
    else:
        return np.NaN

def get_aut_cite_cn(aid):
    if aid in aid_cites_upto_2017:
        return aid_cites_upto_2017[aid]
    else:
        return np.NaN
    
def get_field_vector(doi):
    vec = [0] * len(fields)
    for field in paper_li[doi]['scopus_subjects']:
        if field in field_ix:
            vec[field_ix[field]] = 1
    return vec

def get_disc_vector(doi):
    vec = [0] * len(discs)
    for field in paper_li[doi]['scopus_subjects']:
        if field in discs:
            vec[discs[field]] = 1
    return vec

def clean_field_name(name):
    ans = ''
    for ch in name:
        if ch in ['(', ',', ')', '-', ' ', "'"]:
            ans += '_'
        else:
            ans += ch
    return ans

# make `tid_name_sname` a param.
def get_self_promo(row, tid_name_sname):
    doi, author_name = row['doi'], row['author_name']
    paper = paper_li[doi]
    mtids = [tweet['tweet_id'] for tweet in paper['tweets'] if tweet['tweet_id'] in tid_name_sname]
    # must already be a dict.
    name_tupe = get_author_name(author_name)
    self_promo = False
    mat_tid = ''
    matches = []
    for tid in mtids:
        tname_tupe = tid_name_sname[tid]
        if check_match_author(tname_tupe, name_tupe):
            matches.append((tid, tid_name_sname[tid][0]))
    if len(matches) >= 1:
        self_promo = True
        if len(matches) == 1:
            mat_tid = matches[0][0]
        else:
            tid_scores = {}
            for tid, tw_name in matches:
                tid_scores[tid] = fuzz.ratio(tw_name, author_name)
            mat_tid = sorted(tid_scores.keys(), key=lambda tid: tid_scores[tid], reverse=True)[0]
    return (self_promo, mat_tid)

def get_self_promo_comb_tid(row):
    p_0, p_1, p_2 = row['self_promotion'], row['self_promotion_original'], row['self_promotion_retweet']
    if p_0:
        if p_1:
            return row['matched_tid_original']
        # p_2 must be true
        else:
            return row['matched_tid_retweet']
    else:
        return ''

In [19]:
reg_data = []

cn = 0
for doi in paper_li:
    cn += 1
    if cn%100000 == 0:
        print('processed %d papers...'%cn)
    if doi in doi_author_list:
        # get pub year from paper_li later
        for author in doi_author_list[doi]:
            aid, affis, seq, author_name, pos = author
            name_tupe = get_author_name(author_name)
            if type(name_tupe) == dict:
#                 self_promo, mat = get_self_promo(doi, author_name)
                reg_data.append([doi, author_name, seq, pos, aid, affis])

processed 100000 papers...
processed 200000 papers...
processed 300000 papers...
processed 400000 papers...
processed 500000 papers...
processed 600000 papers...
processed 700000 papers...
processed 800000 papers...
processed 900000 papers...
processed 1000000 papers...
processed 1100000 papers...
processed 1200000 papers...


In [20]:
reg_data = pd.DataFrame(reg_data, columns = ['doi', 'author_name', 'authorship_seq', 'authorship_pos', \
                                             'author_id', 'affiliation_ids'])

In [32]:
reg_data['DV'] = reg_data.apply(lambda row: get_self_promo(row, tid_name_sname_org), axis = 1)
reg_data[['self_promotion_original', 'matched_tid_original']] = pd.DataFrame(reg_data.DV.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['DV'])


In [33]:
np.sum(reg_data['self_promotion_original'] == True)

243313

In [34]:
# use a different `tid_name_sname` for retweets.
reg_data['DV'] = reg_data.apply(lambda row: get_self_promo(row, tid_name_sname_retweet), axis = 1)
reg_data[['self_promotion_retweet', 'matched_tid_retweet']] = pd.DataFrame(reg_data.DV.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['DV'])


In [35]:
np.sum(reg_data['self_promotion_retweet'] == True)

193495

In [36]:
reg_data['self_promotion'] = (reg_data['self_promotion_retweet'] == True) | (reg_data['self_promotion_original'] == True)
reg_data['matched_tid'] = reg_data.apply(lambda row: get_self_promo_comb_tid(row), axis = 1)


In [37]:
np.sum(reg_data['self_promotion'] == True)

363314

In [21]:
# Altmetric data: JAMA: Journal of the American Medical Association
# WOS data: JAMA-JOURNAL OF THE AMERICAN MEDICAL ASSOCIATION
# 2018 version.
journal_impact = {}
for line in yield_one_line(Data_Root+'wos_jcr.csv', delimiter=',', quote=csv.QUOTE_ALL):
    rank, title, cites, jif, eigen = line
    try:
        impact = float(jif)
        ntitle = norm_string(title)
        journal_impact[ntitle] = impact
    except:
        continue

processing /home/haopeng/Data/Promotion/wos_jcr.csv...


In [22]:
len(journal_impact)

11873

In [24]:
affi_rank = {}
affi_country = {}
affi_name = {}
for line in yield_one_line(Data_Root+'Affiliations.txt'):
    affi_id, rank, dname, lat, lon = line[0], line[1], line[3], line[9], line[10]
    affi_rank[affi_id] = int(rank)
    affi_name[affi_id] = dname
    if lat != "" and lon != "":
        lat, lon = float(lat), float(lon)
        res = reverse_geocode.search([(lat, lon)])
        country = res[0]['country']
        affi_country[affi_id] = country

processing /home/haopeng/Data/Promotion/Affiliations.txt...


In [28]:
affi_name

{'20455151': 'Air Liquide',
 '24386293': 'Hellenic National Meteorological Service',
 '32956416': 'Catholic University of the West',
 '35926432': 'Mackay Medical College',
 '37448385': "Chinese People's Public Security University",
 '37712293': 'Madeira Interactive Technologies Institute',
 '42117203': "Queen's College, London",
 '50614327': 'Amec Foster Wheeler',
 '62987803': 'Ukrainian State University of Finance and International Trade',
 '71576600': 'Ural State Mining University',
 '76806421': 'SELEX Sistemi Integrati',
 '78757542': 'University of Newcastle',
 '80046288': 'Walden University',
 '81020251': 'Great Lakes Institute of Management',
 '83776822': 'Shandong Institute of Business and Technology',
 '84117484': 'Collège Boréal',
 '90303723': 'Scientific Council for Government Policy',
 '94975175': 'Des Moines University',
 '95198407': 'HelpAge International',
 '98720382': 'American School of Classical Studies at Athens',
 '98899809': 'National Bank of Belgium',
 '118024222': 

Controls

In [ ]:
reg_data['gender'] = reg_data['author_name'].apply(lambda author_name: name_gender[author_name])

In [25]:
reg_data['num_authors'] = reg_data['doi'].apply(lambda doi: len(doi_author_list[doi]))

In [37]:
reg_data['pub_year'] = 2018
reg_data['author_pub_count'] = reg_data.apply(lambda row: get_aut_pub_cn(row), axis = 1)

In [43]:
reg_data['author_citation'] = reg_data['author_id'].apply(get_aut_cite_cn)

In [33]:
# reg_data['author_feats'] = reg_data['author_id'].map(aid_metric)
# reg_data[['author_rank', 'num_paper', 'num_cite']] = pd.DataFrame(reg_data['author_feats'].values.tolist(), index = reg_data.index)

reg_data['author_rank'] = reg_data['author_id'].apply(get_author_rank)
reg_data['affiliation_name'] = reg_data['affiliation_ids'].apply(get_affi_name)
reg_data['affiliation_cate'] = reg_data['affiliation_ids'].apply(get_affi_cate)
reg_data['affiliation_rank'] = reg_data['affiliation_ids'].apply(get_affi_rank)
reg_data['journal_title'] = reg_data['doi'].apply(lambda doi: paper_li[doi]['journal'] if paper_li[doi]['journal'] != '' else 'unknown')
reg_data['journal_impact'] = reg_data['journal_title'].apply(get_journal_rank)
top_journals = set([jname for jname, num in Counter(reg_data['journal_title']).most_common()[:100] if jname != 'unknown'])
reg_data['top_journal'] = reg_data['journal_title'].apply(get_journal_cate)

In [7]:
aid_rate_pub_cn = {}
for aid, sub in reg_data.groupby('author_id'):
    pub_cn = len(sub)
    aid_rate_pub_cn[aid] = (pub_cn, np.mean(sub['self_promotion']))

In [8]:
reg_data['pub_cn_rate'] = reg_data['author_id'].map(aid_rate_pub_cn)
reg_data[['author_num_papers_in_data', 'author_self_promotion_rate']] = pd.DataFrame(reg_data.pub_cn_rate.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['pub_cn_rate'])

In [9]:
del aid_rate_pub_cn

Discipline

In [85]:
reg_data['field_vector'] = reg_data['doi'].apply(get_field_vector)
reg_data[list(fields.keys())] = pd.DataFrame(reg_data.field_vector.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['field_vector'])

In [86]:
field_name_clean_map = {name: clean_field_name(name) for name in fields}

In [87]:
' + '.join(field_name_clean_map.values())

'Social_Sciences + Materials_Science + Engineering + Chemistry + Biochemistry__Genetics_and_Molecular_Biology + Medicine + Nursing + Agricultural_and_Biological_Sciences + Pharmacology__Toxicology_and_Pharmaceutics + Neuroscience + Business__Management_and_Accounting + Economics__Econometrics_and_Finance + Chemical_Engineering + Physics_and_Astronomy + Computer_Science + Decision_Sciences + Health_Professions + Psychology + Immunology_and_Microbiology + Dentistry + Earth_and_Planetary_Sciences + Environmental_Science + Mathematics + Arts_and_Humanities + Energy + Veterinary + General'

In [88]:
reg_data = reg_data.rename(columns=field_name_clean_map)

In [95]:
reg_data['disc_vector'] = reg_data['doi'].apply(get_disc_vector)
reg_data[list(discs.keys())] = pd.DataFrame(reg_data.disc_vector.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['disc_vector'])

In [102]:
disc_name_clean_map = {name: clean_field_name(name) for name in discs}

In [110]:
reg_data = reg_data.rename(columns=disc_name_clean_map)

Group affiliations

In [99]:
reg_data.affiliation_rank.describe()

count    4.319436e+06
mean     7.588860e+03
std      1.751305e+03
min      4.446000e+03
25%      6.310000e+03
50%      7.273000e+03
75%      8.556000e+03
max      1.923800e+04
Name: affiliation_rank, dtype: float64

In [23]:
np.sum(reg_data.affiliation_rank.isna())

2043973

In [9]:
reg_data['affiliation_rank_cate'] = pd.qcut(reg_data['affiliation_rank'], q=10, labels=False, duplicates='drop')

In [18]:
reg_data.affiliation_rank_cate.value_counts()

0.0    446331
4.0    437319
7.0    433169
2.0    432559
9.0    431545
6.0    431144
8.0    430807
3.0    426949
5.0    426217
1.0    423396
Name: affiliation_rank_cate, dtype: int64

In [25]:
# NaN values are ignored (no cate assigned)
np.sum(reg_data.affiliation_rank_cate.value_counts())

4319436

Group num. of pubs

In [12]:
reg_data.author_pub_count.describe()

count    6.361871e+06
mean     1.126746e+02
std      2.553252e+02
min      1.000000e+00
25%      4.000000e+00
50%      2.500000e+01
75%      1.010000e+02
max      4.871000e+03
Name: author_pub_count, dtype: float64

They added an option duplicates='raise'|'drop' to control whether to raise on duplicated edges or to drop them, which would result in less bins than specified, and some larger (with more elements) than others.

In [6]:
out, bins = pd.qcut(reg_data['author_pub_count'], q=10, labels=False, duplicates='drop', retbins=True)

In [11]:
np.set_printoptions(suppress=True)
bins

array([   1.,    3.,    6.,   13.,   25.,   44.,   77.,  136.,  285.,
       4871.])

In [13]:
reg_data['author_pub_count_cate'] = pd.qcut(reg_data['author_pub_count'], q=10, labels=False, duplicates='drop')

In [17]:
# the first and the second bins are merged.
reg_data.author_pub_count_cate.value_counts()

0.0    1433854
2.0     645255
5.0     645202
7.0     637837
8.0     634473
3.0     628366
6.0     625925
4.0     616789
1.0     494170
Name: author_pub_count_cate, dtype: int64

In [44]:
reg_data.shape

(6363409, 58)

In [45]:
reg_data.to_csv(Data_Root+"reg_data.csv", index=False, header=True, encoding='utf-8')

In [42]:
utype = {'author_id': str, 'matched_tid': 'str', 'matched_tid_original': 'str', 'matched_tid_retweet': 'str'}
reg_data = pd.read_csv(Data_Root+"reg_data.csv", header=0, dtype=utype)

### Gende prediction evaluation

In [85]:
def get_gender_genderize(name, thsh = 0.9):
    if name in name_genderize:
        gender = name_genderize[name]['gender']
        if gender == 'Unknown':
            return 'Unknown'
        else:
            if name_genderize[name]['probability'] >= thsh:
                if gender == 'male':
                    return 'Male'
                if gender == 'female':
                    return 'Female'
            else:
                return 'Unknown'
    else:
        return 'Unknown'
    
def get_gender_ford(name):
    name_tupe = get_author_name(name)
    fname = name_tupe['Fname']
    if len(fname) == 1 or (len(fname) == 2 and fname[-1] == '.'):
        return 'Unknown'
    gender = name_gender[name]
    if gender == 'Unisex':
        return 'Unknown'
    else:
        return gender

In [86]:
api_url = "https://api.genderize.io/?"

In [323]:
payload = {'name': 'Agnes'}
response = requests.get(api_url, params=payload)
eval(response.text)

{'name': 'Agnes', 'gender': 'female', 'probability': 0.98, 'count': 15854}

In [81]:
compare_df = reg_data.sample(n=100)[['doi', 'author_name', 'affiliation_name']]

In [82]:
sample_names = set(compare_df.author_name)

In [83]:
len(sample_names)

100

In [87]:
name_genderize = {}
for name in sample_names:
    name_tupe = get_author_name(name)
    # must be a dict
    fname = name_tupe['Fname']
    payload = {'name': fname}
    response = requests.get(api_url, params=payload)
    if response.status_code == 200:
        res_j = response.json()
        if not res_j['gender']:
            res_j['gender'] = 'Unknown'
        name_genderize[name] = res_j

In [372]:
# compare_df = pd.DataFrame(name_genderize.keys(), columns=['author_name'])

In [88]:
compare_df['genderize'] = compare_df.author_name.apply(lambda name: get_gender_genderize(name, thsh = 0.9))

In [89]:
compare_df['gender_ford'] = compare_df.author_name.apply(lambda name: get_gender_ford(name))

In [90]:
compare_df.genderize.value_counts()

Male       52
Unknown    29
Female     19
Name: genderize, dtype: int64

In [91]:
compare_df.gender_ford.value_counts()

Male       49
Unknown    28
Female     23
Name: gender_ford, dtype: int64

In [92]:
cohen_kappa_score(compare_df.genderize, compare_df.gender_ford)

0.8226664517169111

In [93]:
compare_df['agreement'] = compare_df.apply(lambda row: 'mismatch' if row['genderize'] != row['gender_ford'] else 'match', axis = 1)

In [94]:
compare_df = compare_df.sort_values(by='agreement')

In [95]:
compare_df.to_csv(Data_Root+'gender_label.csv', header=True, index=False)

In [18]:
# add manual verification of gender for each name based on google search.
compare_df = pd.read_csv(Data_Root+'gender_label.csv', header=0)

In [19]:
compare_df = compare_df.loc[compare_df.label!='Unknown']

In [20]:
len(compare_df)

76

In [23]:
print(metrics.classification_report(compare_df.label, compare_df.genderize))

              precision    recall  f1-score   support

      Female       1.00      0.84      0.91        19
        Male       1.00      0.89      0.94        57
     Unknown       0.00      0.00      0.00         0

    accuracy                           0.88        76
   macro avg       0.67      0.58      0.62        76
weighted avg       1.00      0.88      0.94        76



In [22]:
print(metrics.classification_report(compare_df.label, compare_df.gender_ford))

              precision    recall  f1-score   support

      Female       1.00      0.79      0.88        19
        Male       1.00      0.84      0.91        57
     Unknown       0.00      0.00      0.00         0

    accuracy                           0.83        76
   macro avg       0.67      0.54      0.60        76
weighted avg       1.00      0.83      0.91        76

